In [1]:
import torch
import torchvision
import os
import numpy as np
from pathlib import Path
from PIL import Image
from torchvision import transforms, models
import torch.nn as nn
from torch.utils.data import DataLoader, Dataset
import cv2
from src.res_model import ResNetUNet

In [2]:
# GPU 설정
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
print(device)  # Print the device being used

cuda:0


In [3]:
# 모델 설정
unet = ResNetUNet(
    in_channels=1,
    out_channels=2,
    batch_norm=True,
    upscale_mode="bilinear"
)

# 모델 로드
models_folder = Path("models")
model_name = "best_segmentation.pt"
checkpoint = torch.load(models_folder / model_name, map_location=device)
unet.load_state_dict(checkpoint)
unet.to(device)
unet.eval()

/opt/anaconda3/envs/yolo/lib/python3.12/site-packages/torchvision/models/_utils.py:208: UserWarning: The parameter 'pretrained' is deprecated since 0.13 and may be removed in the future, please use 'weights' instead.
  warnings.warn(
/opt/anaconda3/envs/yolo/lib/python3.12/site-packages/torchvision/models/_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet152_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet152_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


ResNetUNet(
  (encoder): ModuleList(
    (0): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
    (1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    (2): ReLU(inplace=True)
    (3): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
    (4): Sequential(
      (0): Bottleneck(
        (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
        (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
        (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
        (relu): ReLU(inplace=True)
        (downsample): Sequential(
          (0): Conv2d(6

In [4]:
# 이미지 리사이즈 및 패딩 함수
def resize_and_pad(image, size=(1024, 1024)):
    old_size = image.size  # old_size is in (width, height) format
    ratio = float(size[0]) / max(old_size)
    new_size = tuple([int(x * ratio) for x in old_size])
    image = image.resize(new_size, Image.LANCZOS)
    new_image = Image.new("L", size)  # "L" 모드로 새 이미지 생성
    new_image.paste(image, ((size[0] - new_size[0]) // 2, (size[1] - new_size[1]) // 2))
    return new_image

# 마스크 후처리 함수
def postprocess_mask(mask, min_area=1000, max_distance=50):
    mask = (mask * 255).astype(np.uint8)
    _, thresh = cv2.threshold(mask, 127, 255, cv2.THRESH_BINARY)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        if cv2.contourArea(contour) < min_area:
            cv2.drawContours(thresh, [contour], -1, 0, thickness=cv2.FILLED)
    new_mask = np.zeros_like(thresh)
    contours, _ = cv2.findContours(thresh, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        cv2.drawContours(new_mask, [contour], -1, 255, thickness=cv2.FILLED)
    kernel = np.ones((5, 5), np.uint8)
    new_mask = cv2.morphologyEx(new_mask, cv2.MORPH_CLOSE, kernel)
    contours, _ = cv2.findContours(new_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    for contour in contours:
        hull = cv2.convexHull(contour)
        cv2.fillPoly(new_mask, [hull], 255)
    contours, _ = cv2.findContours(new_mask, cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
    contours = sorted(contours, key=cv2.contourArea, reverse=True)[:2]
    final_mask = np.zeros_like(new_mask)
    for contour in contours:
        cv2.drawContours(final_mask, [contour], -1, 255, thickness=cv2.FILLED)
    return final_mask / 255.0  # 0과 1 사이의 값으로 정규화

# 이미지 세그멘테이션 함수
def create_segmented_image(img_path):
    origin = Image.open(img_path).convert("L")  # "L" 모드로 변환
    origin = resize_and_pad(origin, (1024, 1024))
    origin_tensor = transforms.functional.to_tensor(origin).unsqueeze(0) - 0.5
    with torch.no_grad():
        origin_tensor = origin_tensor.to(device)
        out = unet(origin_tensor)
        softmax = torch.nn.functional.log_softmax(out, dim=1)
        out = torch.argmax(softmax, dim=1)
        origin_tensor = origin_tensor[0].to("cpu")
        out = out[0].to("cpu")
    processed_mask = postprocess_mask(out.numpy())
    return processed_mask

In [5]:
# 진폐증 판단 모델 설정
pneumonia_model = models.resnet152(pretrained=True)
num_features = pneumonia_model.fc.in_features
pneumonia_model.fc = nn.Linear(num_features, 2)  # Assuming binary classification
pneumonia_model.load_state_dict(torch.load("models/best_pneumonia_model.pth", map_location=device))
pneumonia_model.to(device)
pneumonia_model.eval()

ResNet(
  (conv1): Conv2d(3, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): Bottleneck(
      (conv1): Conv2d(64, 64, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (conv3): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 1), bias=False)
      (bn3): BatchNorm2d(256, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (downsample): Sequential(
        (0): Conv2d(64, 256, kernel_size=(1, 1), stride=(1, 

In [6]:
# 테스트 데이터셋을 처리하고 결과를 평가하는 함수
def evaluate_model(test_dir):
    correct = 0
    total = 0

    for class_name in ["NORMAL", "PNEUMONIA"]:
        class_dir = os.path.join(test_dir, class_name)
        for img_name in os.listdir(class_dir):
            img_path = os.path.join(class_dir, img_name)
            label = 0 if class_name == "NORMAL" else 1

            # 폐영역 분리
            mask = create_segmented_image(img_path)
            mask = torch.tensor(mask).unsqueeze(0).unsqueeze(0).float().to(device)

            # 원본 이미지 로드 및 변환
            image = Image.open(img_path).convert("RGB")
            image = resize_and_pad(image, (1024, 1024))  # 원본 이미지를 1024x1024로 리사이즈 및 패딩
            transform = transforms.Compose([
                transforms.Grayscale(num_output_channels=3),
                transforms.ToTensor()
            ])
            image = transform(image).unsqueeze(0).to(device)

            # 마스크를 적용하여 폐 영역만 남김
            mask_expanded = mask.expand_as(image)
            masked_image = image * mask_expanded

            # 진폐증 판단
            with torch.no_grad():
                outputs = pneumonia_model(masked_image)
                _, predicted = torch.max(outputs, 1)

            total += 1
            correct += (predicted == label).sum().item()

    accuracy = correct / total
    print(f'Correct: {correct}, Total: {total}, Accuracy: {accuracy:.4f}')

In [7]:
# 테스트 데이터셋 평가
evaluate_model('input/chest_xray/test')

Correct: 540, Total: 624, Accuracy: 0.8654
